In [7]:
# 加载各种库
import torch
import os
import pickle
import torch.nn as nn
from torchvision.transforms import ToTensor
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import CIFAR10
import numpy as np
import random
from torch.optim.lr_scheduler import CosineAnnealingLR  # 实现cos函数式的变化
# 加载支持函数和类
from support_code.load_dataset import get_data_loader
from support_code.LeNet import LeNet
# 设置设备和种子
device_cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 设置随机种子
seed = 10  # 你可以选择任何整数作为种子值

# 设置PyTorch随机种子
torch.manual_seed(seed)

# 设置CUDA随机种子（如果使用GPU）
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 设置Numpy随机种子
np.random.seed(seed)

# 设置Python随机种子
random.seed(seed)



In [8]:
def get_scale(scale, writer,  
            device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), train_loader=None, valid_loader=None, test_loader=None, 
            channel=3,
            num_classes=10,
            im_size=(32, 32),
            # 可能需要的超参数
            momentum = 0.9,
            weight_decay = 0.0005,
            initial_lr = 0.1,
            num_epochs = 100,
            save_dir = "logs-LeNet-0913"):

  # -------------------------------------------------创建模型实例并将其移至 GPU -----------------------------------------------------
  model = LeNet(channel=channel, num_classes=num_classes)
  model.to(device)
  # 确保文件夹存在，如果不存在则创建它
  os.makedirs(save_dir, exist_ok=True)
  # 定义损失函数/优化器/学习率
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=initial_lr, weight_decay=weight_decay, momentum=momentum)
  scheduler = CosineAnnealingLR(optimizer, T_max=10)

  # 记录训练过程中的准确率和损失
  tag_loss = "train_loss_{}".format(scale)
  tag_acc = "train_accuracy_{}".format(scale)

  # 训练循环...
  final_train_loss = 0
  final_train_acc = 0
  best_train_acc = 0.0  # 用于跟踪一个scale也就是一个完整的train的过程中的最佳准确率

  # 验证步骤...
  # 记录验证的次数
  valid_correct = 0
  valid_total = 0
  best_valid_acc = 0.0
  val_loss = "val_loss_{}".format(scale)
  val_acc = "val_accuracy_{}".format(scale)
  acc = 0.0
  loss = 1.0

  for epoch in range(num_epochs):

    # ----------------------------首先是训练过程----------------------------
    train_correct = 0
    train_total = 0
    for batch_idx, (imgs, targets) in enumerate(train_loader):
      imgs, targets = imgs.to(device), targets.to(device)
      outputs = model(imgs)
      loss = criterion(outputs, targets)
      _, predicted = torch.max(outputs, 1)
      train_total += targets.size(0)
      train_correct += (predicted == targets).sum().item()
      optimizer.zero_grad() # W_new = W_old + \gamma \times \deta W_old,这里相当于先要求\gamma 为零矩阵
      loss.backward()   # 使用backward得到了\deta W_old
      optimizer.step()  # 选择合适的\gamma 得到了W_new
      scheduler.step()  # \gamma 事实上是一个依赖于learning_rate的一个矩阵函数,这里更新了learning_rate,而且learning_rate的更新是依赖于epoch的

    acc = train_correct / train_total
    writer.add_scalar(tag_loss, loss.item(), epoch)
    writer.add_scalar(tag_acc, acc, epoch)
    # 更新模型权重并记录到Tensorboard
    for name, param in model.named_parameters():
      writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
    final_train_acc = train_correct / train_total
    # 更新最佳准确率,以及对应的loss
    if final_train_acc >= best_train_acc:
      best_train_acc = final_train_acc
      best_train_loss = loss.item()
    final_train_loss = loss.item()
    print("Epoch [{}/{}], Final Train Loss: {:.4f}, Final Train Accuracy: {:.2f}%, Best Train Accuracy: {:.2f}, Loss to best acc:{:.2f}%".format(epoch+1, num_epochs, final_train_loss, final_train_acc * 100, best_train_acc * 100, best_train_loss),"\n")
    # 记录学习率(learning_rate)的更新情况
    writer.add_scalar("learning_rate",optimizer.param_groups[0]['lr'])

    # -------------------------然后进入了验证步骤---------------------------------
    for batch_idx, (imgs, targets) in enumerate(valid_loader):
      imgs, targets = imgs.to(device), targets.to(device)
      outputs = model(imgs)
      loss = criterion(outputs, targets)
      _, predicted = torch.max(outputs, 1)
      valid_total += targets.size(0)
      valid_correct += (predicted == targets).sum().item()
      acc = valid_correct / valid_total

    writer.add_scalar(val_loss, loss.item(), epoch)
    writer.add_scalar(val_acc, acc, epoch)
    # 记录valid中的最终情况
    final_valid_acc = acc
    final_valid_loss = loss
    # 更新最佳准确率,以及对应的loss
    if final_valid_acc >= best_valid_acc:
      best_valid_acc = final_valid_acc
      best_valid_loss = loss.item()
    final_valid_loss = loss.item()
    print("Epoch [{}/{}], Final Valid Loss: {:.4f}, Final Valid Accuracy: {:.2f}%, Best Valid Accuracy: {:.2f}%, Loss to best Valid accuracy:{:.2f}".format(epoch+1, num_epochs, final_valid_loss, final_valid_acc * 100, best_valid_acc * 100, best_valid_loss),"\n")


  # 记录train过程中的最终情况
  writer.add_scalar("final_train_loss", final_train_loss, scale)
  writer.add_scalar("final_train_acc", final_train_acc, scale)
  writer.add_scalar("best_train_acc", best_train_acc, scale)
  writer.add_scalar("best_train_loss", best_train_loss, scale)
  # 记录valid的最终情况
  writer.add_scalar("final_valid_loss", final_valid_loss, scale)
  writer.add_scalar("final_valid_acc", final_valid_acc, scale)
  writer.add_scalar("best_valid_acc", best_valid_acc, scale)
  writer.add_scalar("best_valid_loss", best_valid_loss, scale)

  # 保存模型参数
  # 构建完整的文件路径
  filename = os.path.join(save_dir, "scale={}.pth".format(scale))
  # 保存模型
  torch.save(model.state_dict(), filename)
  print("模型参数已保存为{}".format(filename), '\n')

  # 完成各种工作，记录结果...
  normal_dict = {
      "训练的模型的保留路径": filename,
      "在测试集中的最终损失": final_valid_loss,
      "在测试集中的最终损失":best_valid_loss,
      "在测试集中的最终准确率": final_valid_acc,
      "在测试集中的最佳准确率": best_valid_acc,
      "在训练集中的最终准确率": final_train_acc,
      "在训练集中的最佳准确率": best_train_acc,
      "在训练集中的最终损失": final_train_loss,
      "在训练集中的最佳损失":best_train_loss
  }
  return normal_dict

In [9]:
# 定义初始的scale范围
scale_min = 0.1
scale_max = 0.9
scale_list = [round(x * 0.1, 1) for x in range(int(scale_min * 10), int(scale_max * 10) + 1)]
# print(scale_list)
my_list = []
# 添加tensorboard
writer_new = SummaryWriter("logs-LeNet-0913")

for scale in scale_list:
  train_loader, valid_loader, test_loader = get_data_loader(scale, valid_scale=0.1)
  dic = {}
  print("--------------------开始scale={}的情况---------------------".format(scale), "\n")
  dic['scale={}'.format(scale)] = get_scale(scale=scale, writer=writer_new, train_loader=train_loader, valid_loader=valid_loader,save_dir = "logs-LeNet-0913", num_epochs=200)
  print(dic['scale={}'.format(scale)])
  my_list.append(dic)

writer_new.close()

# -------------------------------------保存列表,因为我发现notebook不是很可靠-----------------------------
# 指定保存文件的文件夹和文件名
folder_path = "logs-LeNet-0913"
file_name = "dic_scale_512.pkl"

# 确保文件夹存在，如果不存在则创建它
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 组合文件夹路径和文件名
dic_file_path = os.path.join(folder_path, file_name)

with open(dic_file_path, "wb") as file:
  pickle.dump(dic, file=file)

print("字典保存在文件中")

Files already downloaded and verified
Files already downloaded and verified
--------------------开始scale=0.1的情况--------------------- 

Epoch [1/200], Final Train Loss: 2.1605, Final Train Accuracy: 16.21%, Best Train Accuracy: 16.21, Loss to best acc:2.16% 

Epoch [1/200], Final Valid Loss: 2.2354, Final Valid Accuracy: 16.00%, Best Valid Accuracy: 16.00%, Loss to best Valid accuracy:2.24 

Epoch [2/200], Final Train Loss: 2.0804, Final Train Accuracy: 20.26%, Best Train Accuracy: 20.26, Loss to best acc:2.08% 

Epoch [2/200], Final Valid Loss: 2.2882, Final Valid Accuracy: 14.95%, Best Valid Accuracy: 16.00%, Loss to best Valid accuracy:2.24 

Epoch [3/200], Final Train Loss: 2.0522, Final Train Accuracy: 22.37%, Best Train Accuracy: 22.37, Loss to best acc:2.05% 

Epoch [3/200], Final Valid Loss: 2.2383, Final Valid Accuracy: 14.97%, Best Valid Accuracy: 16.00%, Loss to best Valid accuracy:2.24 

Epoch [4/200], Final Train Loss: 1.9623, Final Train Accuracy: 24.29%, Best Train Accurac